In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')

In [ ]:
train = pd.read_csv(r'../input/loan-prediction-analytics-vidhya/train_ctrUa4K.csv')
test = pd.read_csv(r'../input/loan-prediction-analytics-vidhya/test_lAUu6dG.csv')

In [ ]:
train.shape, test.shape

In [ ]:
train['source'] = 'train'
test['source'] = 'test'
data = pd.concat([train,test],ignore_index=True)

In [ ]:
data.shape

In [ ]:
data.dtypes

In [ ]:
data.isnull().sum()

In [ ]:
# filling missing values of categorical variables with mode

data['Gender'].fillna(data['Gender'].mode()[0], inplace=True)

data['Married'].fillna(data['Married'].mode()[0], inplace=True)

data['Dependents'].fillna(data['Dependents'].mode()[0], inplace=True)

data['Self_Employed'].fillna(data['Self_Employed'].mode()[0], inplace=True)

data['Loan_Amount_Term'].fillna(data['Loan_Amount_Term'].mode()[0], inplace=True)

data['Credit_History'].fillna(data['Credit_History'].mode()[0], inplace=True)

In [ ]:
# filling missing values of continuous variables with mean
data['LoanAmount'].fillna(data['LoanAmount'].mean(), inplace=True)

In [ ]:
data.isnull().sum()

In [ ]:
# converting the categories into numbers using map function
data['Gender'] = data['Gender'].map({'Male': 0, 'Female': 1})
data['Married'] = data['Married'].map({'No': 0, 'Yes': 1})
data['Dependents'] = data['Dependents'].map({'0': 0, '1': 1, '2': 2, '3+': 3})
data['Education'] = data['Education'].map({'Graduate': 1, 'Not Graduate': 0})
data['Self_Employed'] = data['Self_Employed'].map({'No': 0, 'Yes': 1})
data['Property_Area'] = data['Property_Area'].map({'Rural': 0, 'Semiurban': 1, 'Urban': 2})
data['Loan_Status'] = data['Loan_Status'].map({'N': 0, 'Y': 1})

In [ ]:
data.head()

In [ ]:
# bringing variables in the range 0 to 1
data['Dependents']=(data['Dependents']-data['Dependents'].min())/(data['Dependents'].max()-data['Dependents'].min())

In [ ]:
train_modified = data.loc[data['source']=='train']
test_modified = data.loc[data['source']=='test']

train_modified.drop('source',axis=1,inplace=True)
test_modified.drop(['source','Loan_Status'],axis=1,inplace=True)

In [ ]:
train_modified.drop('Loan_ID',axis=1,inplace=True)

In [ ]:
# applying for loop to bring all the variables in range 0 to 1

for i in train_modified.columns[1:]:
    train_modified[i] = (train_modified[i] - train_modified[i].min()) / (train_modified[i].max() - train_modified[i].min())

In [ ]:
X = train_modified.drop('Loan_Status',axis=1)
y = train_modified['Loan_Status']

In [ ]:
X.shape, y.shape

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test , y_train,y_test = train_test_split(X,y,stratify=train_modified['Loan_Status'],random_state=10,test_size=0.2)

In [ ]:
from keras.models import Sequential
from keras.layers import InputLayer, Dense

In [ ]:
# defining input neurons
input_neurons = X_train.shape[1]

In [ ]:
# number of output neurons
# since loan prediction is a binary classification problem, we will have single neuron in the output layer 
# define number of output neurons
output_neurons = 1

In [ ]:
# number of hidden layers and hidden neurons
# It is a hyperparameter and we can pick the hidden layers and hidden neurons on our own
# define hidden layers and neuron in each layer
number_of_hidden_layers = 2
neuron_hidden_layer_1 = 10
neuron_hidden_layer_2 = 5

activation function of different layers

for now I have picked relu as an activation function for hidden layers, you can change it as well
since it is a binary classification problem, I have used sigmoid activation function in the final layer

In [ ]:
# defining the architecture of the model
model = Sequential()
model.add(InputLayer(input_shape=(input_neurons,)))
model.add(Dense(units=neuron_hidden_layer_1, activation='relu'))
model.add(Dense(units=neuron_hidden_layer_2,activation='relu'))
model.add(Dense(units=output_neurons, activation='sigmoid'))

In [ ]:
# summary of the model
model.summary()

In [ ]:
# number of parameters between input and first hidden layer

input_neurons*neuron_hidden_layer_1

In [ ]:
# number of parameters between input and first hidden layer
# adding the bias for each neuron of first hidden layer

input_neurons*neuron_hidden_layer_1 + 10

In [ ]:
# number of parameters between first and second hidden layer

neuron_hidden_layer_1*neuron_hidden_layer_2 + 5

In [ ]:
# number of parameters between second hidden and output layer

neuron_hidden_layer_2*output_neurons + 1

In [ ]:
# compiling the model

# loss as binary_crossentropy, since we have binary classification problem
# defining the optimizer as adam
# Evaluation metric as accuracy

model.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [ ]:
# training the model

# passing the independent and dependent features for training set for training the model

# validation data will be evaluated at the end of each epoch

# setting the epochs as 50

# storing the trained model in model_history variable which will be used to visualize the training process

model_history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50)

In [ ]:
# getting predictions for the validation set
prediction = model.predict_classes(X_test)

In [ ]:
# calculating the accuracy on validation set
from sklearn.metrics import accuracy_score
accuracy_score(y_test, prediction)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
# summarize history for loss
plt.plot(model_history.history['loss'])
plt.plot(model_history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
# summarize history for accuracy
plt.plot(model_history.history['accuracy'])
plt.plot(model_history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()